In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
class carProperties(object):
    def __init__(self, url):
        self.url = url
        self._soup = None
        self.title = None
        self.price = None
        self.overview_dict = dict()
        self.consupmtion_attributes_dict = dict()

        self.parse()
        self.build()

    def parse(self):
        response = requests.get(self.url)
        self._soup = BeautifulSoup(response.text, 'html.parser')

    def get_title(self):
        self.title = self._soup.findAll('h1')[0].text

    def get_price(self):
        self.price = self._soup.body.findAll(text='Preis')[0].parent.parent.findAll('p')[1].text

    def get_overview(self):
        self.overview_dict = {l.findChild('dt').text.replace(':','') :l.findChild('dd').text for l in self._soup.findAll('dl')}

    def get_consumption(self):
        consupmtion_attributes_list =[
            'Verbrauch außerorts', 'Verbrauch kombiniert', 'Zugrunde liegende Treibstoffart',
            'CO2-Emissionen', 'Umweltplakette', 'Abgasnorm', 'Jahressteuer']

        self.consupmtion_attributes_dict=dict()

        for k in consupmtion_attributes_list:
            try:
                self.consupmtion_attributes_dict[k] = self._soup.body.findAll(text=k)[0].parent.parent.findAll('div')[1].text
            except:
                print(f'Attribute {k} not found')

    def build(self):
        self.get_title()
        self.get_price()
        self.get_overview()
        self.get_consumption()


In [4]:
url = 'https://www.mahag.de/gebrauchte/fahrzeugsuche/?ahPg=show.vehicle.html&ahId=4317829'

new_car = carProperties(url)


Attribute CO2-Emissionen not found


In [5]:
for l in new_car._soup.findAll("div", {"class": "container-equipment"}):
    print(type(l),l.text)

<class 'bs4.element.Tag'> 
Übersicht Ausstattung


 ABS
 Antischlupfregelung
 Beifahrerairbag
 Berganfahrassistent
 Bluetooth
 Bordcomputer
 Freisprechanlage


 Handyvorbereitung
 ISOFIX vorbereitet
 Katalysator
 Klimaanlage
 Kopfairbag
 Lichtassistent
 MP3


 Multifunktionsdisplay
 Radio
 Reifendruckkontrolle
 Seitenairbag
 Servolenkung
 Sitzheizung
 Stabilitätsprogramm


 Start-/Stop-Automatik
 Tagfahrlicht
 USB-Schnittstelle
 Wegfahrsperre
 Zentralverriegelung
 el. Fensterheber
 elektr. Differentialsperre





In [167]:
soup.body.findAll(text='Verbrauch innerorts')[0].parent.parent.findAll('div')[1].text

'6,0 l/100km'

In [33]:

for l1 in l.children:
    print(l1,type(l1))
    print('-'*10)

# new_car._soup.body.findAll(text='Bordcomputer')
# .parent.parent.findAll('div')[1].text


 <class 'bs4.element.NavigableString'>
----------
<h4 class="uk-margin-large-top">Übersicht Ausstattung</h4> <class 'bs4.element.Tag'>
----------

 <class 'bs4.element.NavigableString'>
----------
<div class="uk-grid">
<div class="uk-width-mini-1-1 uk-width-small-1-2 uk-width-large-1-4">
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> ABS<br/>
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> Antischlupfregelung<br/>
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> Beifahrerairbag<br/>
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> Berganfahrassistent<br/>
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> Bluetooth<br/>
<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/> Bordcomputer<br/>
<img src="//homepage.autohausen.d

<img src="//homepage.autohausen.de/demo2.1/static/default/images/list-images/ok-3ebdcc.png"/>